In [1]:
import yfinance as yf

df= yf.download("TCS.NS", start="2021-07-01", end="2022-01-28")
import pandas as pd
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)
df.columns.name = None 
dataset=df.drop(columns=['Volume'])
dataset

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open
Date,,,,
2021-07-01,3117.359619,3136.484525,3108.823472,3132.752836
2021-07-02,3102.386230,3115.680372,3074.165333,3107.563994
2021-07-05,3098.234863,3147.679744,3080.509340,3117.826231
2021-07-06,3043.472168,3104.765112,3037.688096,3090.771278
2021-07-07,3055.273438,3062.503584,3024.160620,3055.320129
...,...,...,...,...
2022-01-20,3590.828125,3678.521402,3576.235985,3669.137419
2022-01-21,3597.349609,3614.287743,3538.793652,3572.482056
2022-01-24,3539.028076,3612.504471,3509.703133,3603.449020


In [2]:
#column='Close'

In [3]:
from sklearn.preprocessing import MinMaxScaler
Ms=MinMaxScaler()
data1=Ms.fit_transform(dataset)
print('Len:',data1.shape)

Len: (144, 4)


In [4]:
len(data1)

144

In [5]:
training_size=round(len(data1)* 0.85)
print(training_size)
x_train=data1[:training_size]
x_test=data1[training_size:]
print("x_train length:",x_train.shape)
print("x_test length:",x_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("x_train length:",y_train.shape)
print("x_test length:",y_test.shape)

122
x_train length: (122, 4)
x_test length: (22, 4)
x_train length: (122, 4)
x_test length: (22, 4)


In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [39]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [40]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [46]:
listt=["Close","High"]

In [47]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High']
Order = 1
AIC:  -13.382310641471717
BIC:  -13.239096791296243

Order = 2
AIC:  -13.376747404213688
BIC:  -13.136729996459785

Order = 3
AIC:  -13.304814638384652
BIC:  -12.966908182968963

Order = 4
AIC:  -13.244342625977417
BIC:  -12.807441021662134

Order = 5
AIC:  -13.181502932016173
BIC:  -12.644478928152466

Order = 6
AIC:  -13.112888766341905
BIC:  -12.474593407154607

Order = 7
AIC:  -13.089189788353616
BIC:  -12.348451839116237

Order = 8
AIC:  -13.040747763732965
BIC:  -12.19637311815682

Order = 9
AIC:  -12.977385951780331
BIC:  -12.028157019915355

Order = 10
AIC:  -12.900921430636217
BIC:  -11.845596506554077



In [48]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.287943,1,28
1,"[Close, High, Open]",0,0.288736,1,28
2,"[Close, High]",0,0.253608,2,28
